In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import sklearn.neighbors._base
# import sys
# sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.ensemble import RandomForestClassifier


In [4]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced


AttributeError: module 'keras.utils' has no attribute 'Sequence'

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [ ]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=0)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

In [ ]:
df.loc[df["debt_settlement_flag"] == "y"]

# Split the Data into Training and Testing

In [7]:
# Create our features
#X = # YOUR CODE HERE

# Create our target
#y = # YOUR CODE HERE
X = df.drop(columns='loan_status')
X = pd.get_dummies(X)

y = df.loc[:,target].copy()

# x_ensm = [i for i in df.columns if i not in ('ID', 'debt_settlement_flag')]
# X = df[x_ensm]
# y = df['debt_settlement_flag']
# X = pd.get_dummies(X)

NameError: name 'df' is not defined

In [ ]:
X.describe()

In [ ]:
y= pd.DataFrame(y)

In [6]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train,X_test,y_train,y_test = train_test_split(X, y, random_state=1)

NameError: name 'X' is not defined

In [14]:
print(X_train)

       loan_amnt  int_rate  installment  annual_inc    dti  delinq_2yrs  \
3903      1600.0    0.0881        50.74     35964.0  24.12          0.0   
28390     9000.0    0.0756       280.21     41000.0  16.89          1.0   
15470    10000.0    0.1033       214.10    112000.0  17.75          0.0   
4279     36000.0    0.1033      1167.21    120000.0  19.95          0.0   
57514    18000.0    0.0881       570.81     51000.0  19.11          1.0   
...          ...       ...          ...         ...    ...          ...   
49100     8400.0    0.0646       257.30     37992.0  33.16          0.0   
20609    30000.0    0.0646       918.93    217000.0  19.56          0.0   
21440    20000.0    0.1180       442.88     35000.0  16.19          0.0   
50057    11625.0    0.1447       273.34     30000.0  18.96          0.0   
5192     20000.0    0.1033       428.20     60000.0  59.56          0.0   

       inq_last_6mths  open_acc  pub_rec  revol_bal  ...  issue_d_19-Mar  \
3903              0.0  

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [15]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier




In [16]:
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 

In [17]:
# from sklearn.datasets import make_classification

# X, y = make_classification(n_samples=1000, n_classes=3,
#                            n_informative=4, weights=[0.2, 0.3, 0.5],
#                            random_state=0)


In [18]:
print(X.shape)

(68817, 95)


In [19]:
print(y.shape)

(68817, 1)


In [20]:
brf_model.fit(X_train, y_train)


ValueError: object of too small depth for desired array

In [21]:
predictions = brf_model.predict(X_test)

predictions

IndexError: list index out of range

In [22]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
acc_score = balanced_accuracy_score(y_test, predictions)


NameError: name 'predictions' is not defined

In [25]:
acc_score

NameError: name 'acc_score' is not defined

In [26]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,predictions)


NameError: name 'predictions' is not defined

In [27]:
cm

NameError: name 'cm' is not defined

In [28]:
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

NameError: name 'cm' is not defined

In [29]:
# Print the imbalanced classification report
# YOUR CODE HERE
print("Confusion Matrix")
display(cm_df)
#print(f"Accuracy Score : {acc_score}")
display(acc_score)
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))



Confusion Matrix


NameError: name 'cm_df' is not defined

In [30]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE

importances = brf_model.feature_importances_
importances

sorted(zip(brf_model.feature_importances_, X.columns), reverse=False)


NameError: name 'brf_model' is not defined

### Easy Ensemble AdaBoost Classifier

In [31]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier 

eec = EasyEnsembleClassifier(n_estimators=100,random_state=1)





ModuleNotFoundError: No module named 'sklearn.neighbors.base'

In [32]:
eec.fit(X_train, y_train)

NameError: name 'eec' is not defined

In [70]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
predictions = eec.predict(X_test)
balanced_accuracy_score(y_test,predictions)

0.5

In [73]:
# Display the confusion matrix
# YOUR CODE HERE
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,101,0
Actual 1,17104,0


In [75]:
# Print the imbalanced classification report
# YOUR CODE HERE
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      1.00      0.00      0.01      0.00      0.00       101
   low_risk       0.00      0.00      1.00      0.00      0.00      0.00     17104

avg / total       0.00      0.01      0.99      0.00      0.00      0.00     17205

